# Load Libraries

In [1]:
import torch, sys
from datetime import datetime
# from random import randint
from numpy import repeat
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

import digits

# Load Model

In [2]:
pretrained_model = 'openai-community/gpt2-xl'
# pretrained_model = 'EleutherAI/gpt-neo-1.3B'
# pretrained_model = 'perplexity-ai/r1-1776' ## Bloody large...
# pretrained_model = 'meta-llama/Llama-3.2-3B-Instruct' ## Doesn't seem to work well

pretrained_model_name = pretrained_model.split("/")[-1]

tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
# tokenizer.pad_token_id = tokenizer.eos_token_id

model     = AutoModelForCausalLM.from_pretrained(pretrained_model).to('mps')

# Main functional loop

In [3]:
stopwords = [0, 1, 6, 13, 30, 3548, 3228, 50256] 
# double quotation mark, single quotation mark,
# exclamation mark, fullstop, 
# question mark, two question marks, 
# two exclamation marks, EOS.

prompts = ["The model exhibits signs of prediction envy",
           "Humans do not simply predict the next word.",
           "Longing for an unlikely outcome,",
           "Do you enjoy listening to a detuned piano?",
           "The mirrors always break in the same ways.",
           "This tongue is inevitably forked.",
           "This is the cure for prediction envy:"
          ]

In [4]:
min_tokens = 88

for prompt in prompts:
    print(prompt)
    generated = prompt
    keep_generating = True
    
    for idx, d in enumerate(digits.PHI):
    
        if keep_generating:
            encoded = tokenizer(generated, return_tensors="pt").to("mps")
            
            # Get output from decoder.
            outputs = model.generate(**encoded,
                                     return_dict_in_generate=True, 
                                     output_scores=True,
                                     # output_hidden_states=True,
                                     max_new_tokens=1,
                                     do_sample=False,
                                     pad_token_id=50256 #128001
                                     )
            
            # Convert to probabilities.
            probs = torch.nn.functional.softmax(outputs.scores[0], dim=1).cpu()
    
            # Roll my own lookup.
            probs = pd.DataFrame(probs.tolist()[0])
            probs.columns = ['probability']
            probs.sort_values(ascending=False, by='probability', inplace=True)
            
            # Use the digits of pi!
            token_id = probs.index[int(d)]
            top10 = probs.head(10).copy()
        
            if idx > min_tokens-1:
                if any(item in top10.index for item in stopwords):
                    # take the least likely ending token.
                    token_id = [i for i in top10.index if i in stopwords][-1]
                    # and stop generating more text.
                    keep_generating = False
                    
            generated = generated + tokenizer.decode(token_id, 
                                                     skip_special_tokens=True, 
                                                     clean_up_tokenization_spaces=True)
        
            # Update current token count.
            sys.stdout.flush()
            sys.stdout.write(f"\r{idx+1}")
    
    # Writing to file.
    current_time = datetime.now().strftime("%d%m%y")
    filename = f'generations/Phi_GoldenRatio_Digits_{prompt}_{pretrained_model_name}_maxTokens-{min_tokens}_{current_time}.txt'
    
    with open(filename, 'w') as out:
        out.write(f"{pretrained_model_name} \n" + generated )

The model exhibits signs of prediction envy
95Humans do not simply predict the next word.
90Longing for an unlikely outcome,
90Do you enjoy listening to a detuned piano?
92The mirrors always break in the same ways.
96This tongue is inevitably forked.
90This is the cure for prediction envy:
92